In [1]:
# imports
import os
import sys
import webbrowser
import matplotlib.pyplot as plt

import pymodelica
from pymodelica import compile_fmu
from pyfmi import load_fmu
from oct.steadystate.nlesol import FMUProblem, Solver, SolverOptions

# paths
homeDir = os.environ['HOMEPATH']
jmodDir = os.environ['JMODELICA_HOME']
dodyDir = os.path.join(homeDir, "Documents", "Dymola")

# give the path to directory where package.mo is stored
moLibs = [os.path.join(dodyDir, "ModelicaStandardLibrary", "Modelica"),
          os.path.join(dodyDir, "ModelicaStandardLibrary", "ModelicaServices"),
          os.path.join(dodyDir, "ModelicaStandardLibrary", "ModelicaTest"),
          os.path.join(dodyDir, "MWE", "TestMWE"),
         ]


print(sys.version)
print pymodelica.__version__
print("="*76)
print(os.getcwd())
for moLib in moLibs:
    if moLib.endswith("mo"):
        print(str(os.path.isfile(moLib)) + ": " + moLib)
    else:        
        print(str(os.path.isfile(os.path.join(moLib, "package.mo"))) + ": " + os.path.join(moLib, "package.mo"))

2.7.13 (v2.7.13:a06454b1afa1, Dec 17 2016, 20:53:40) [MSC v.1500 64 bit (AMD64)]
OCT-dev-cw-2718-r12038_JM-dev-cw-2718-r14269
C:\Users\matthis.thorade\Documents\git\jupyterNotebooks\OCT
True: \Users\matthis.thorade\Documents\Dymola\ModelicaStandardLibrary\Modelica\package.mo
True: \Users\matthis.thorade\Documents\Dymola\ModelicaStandardLibrary\ModelicaServices\package.mo
True: \Users\matthis.thorade\Documents\Dymola\ModelicaStandardLibrary\ModelicaTest\package.mo
True: \Users\matthis.thorade\Documents\Dymola\MWE\TestMWE\package.mo


In [2]:
# compile model to fmu
model_name = 'TestMWE.BranchingDynamicPipes'
my_fmu = compile_fmu(model_name, 
                     moLibs, 
                     compiler_log_level='verbose:compilerlog.txt',
                     compiler_options={"generate_html_diagnostics":"true"} , 
                    )
print(my_fmu)

C:\Users\matthis.thorade\Documents\git\jupyterNotebooks\OCT\.\TestMWE_BranchingDynamicPipes.fmu


In [3]:
# open html diagnostics index
url = os.path.join(os.getcwd(), model_name.replace('.','_') + "_html_diagnostics", "index.html")
print(str(os.path.isfile(url)) + ": " + url)
webbrowser.open(url,new=2)

True: C:\Users\matthis.thorade\Documents\git\jupyterNotebooks\OCT\TestMWE_BranchingDynamicPipes_html_diagnostics\index.html


True

In [4]:
# simulate the fmu and store results
myModel = load_fmu(my_fmu)

opts = myModel.simulate_options()
opts['solver'] = "CVode"
opts['ncp'] = 500
opts['result_handling']="file"
opts["CVode_options"]['discr'] = 'BDF'
opts['CVode_options']['iter'] = 'Newton'
opts['CVode_options']['maxord'] = 5
opts['CVode_options']['atol'] = 1e-6
opts['CVode_options']['rtol'] = 1e-6

res = myModel.simulate(start_time=0, final_time=1, options=opts)

Final Run Statistics: --- 

 Number of steps                                 : 501
 Number of function evaluations                  : 505
 Number of Jacobian evaluations                  : 9
 Number of function eval. due to Jacobian eval.  : 0
 Number of error test failures                   : 0
 Number of nonlinear iterations                  : 501
 Number of nonlinear convergence failures        : 0
 Number of state function evaluations            : 502

Solver options:

 Solver                   : CVode
 Linear multistep method  : BDF
 Nonlinear solver         : Newton
 Linear solver type       : DENSE
 Maximal order            : 5
 Tolerances (absolute)    : 1e-06
 Tolerances (relative)    : 1e-06

Simulation interval    : 0.0 - 1.0 seconds.
Elapsed simulation time: 1.9177959 seconds.


In [ ]:
# plot all states
time = 'time'
state_vars = myModel.get_states_list()
print(list(state_vars))

x = res[time]
for varname in state_vars:
    y = res[varname]
    plt.plot(x, y)
    plt.xlabel('Time (s)')
    plt.ylabel(varname)
    plt.savefig(varname.replace('.','_')+'.pdf')
    plt.show()